<a href="https://colab.research.google.com/github/hyunj941031/ds-sa-cp2/blob/main/models/BERT4Rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install python-box

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.backends.cudnn as cudnn

from box import Box

import warnings

warnings.filterwarnings(action='ignore')
torch.set_printoptions(sci_mode=True)

# 1. 학습 parameter 설정

In [4]:
config = {
    'data_path' : '/content/drive/MyDrive/fashion_campus_dataset',
    
    'sequence_len' : 50,
    'mask_prob' : 0.3, # cloze task
    'random_seed' : 123,

    'num_layers' : 2, # block 수 (encoder layer 수)
    'hidden_units' : 50, # Embedding size
    'num_heads' : 2, # Multi-head layer 수 (병렬처리), hidden_units를 나눴을 때 나누어 떨어지게게
    'dropout' : 0.15, # dropout의 비율

    'epoch' : 5,
    'patience' : 5,
    'batch_size' : 256,
    'lr' : 0.001,

    'num_epochs' : 10,
    'num_workers' : 4,
    'val_data' : 3,
    'delete_data' : False,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = Box(config)

# 2. 데이터 전처리

In [5]:
class MakeSequenceDataSet():
    """
    SequenceData 생성
    """
    def __init__(self, config):
        self.config = config
        self.df = pd.read_csv(os.path.join(self.config.data_path, 'user_item.csv'))
        if config['delete_data']:
            self.df = self.delete_ones()

        self.item_encoder, self.item_decoder = self.generate_encoder_decoder('itemId')
        self.user_encoder, self.user_decoder = self.generate_encoder_decoder('userId')
        self.num_items, self.num_users = len(self.item_encoder), len(self.user_encoder)

        self.df['item_idx'] = self.df['itemId'].apply(lambda x : self.item_encoder[x] + 1)
        self.df['user_idx'] = self.df['userId'].apply(lambda x : self.user_encoder[x])
        self.df = self.df.sort_values(['user_idx', 'timestamp']) # 시간에 따른 정렬
        self.user_train, self.user_valid = self.generate_sequence_data()

    def generate_encoder_decoder(self, col:str) -> dict:
        '''
        encoder, decoder 생성

        Args:
            col (str): 생성할 columns 명
        Return:
            dict: 생성된 user encoder, decoder
        '''

        encoder = {}
        decoder = {}
        ids = self.df[col].unique()

        for idx, _id in enumerate(ids):
            encoder[_id] = idx
            decoder[idx] = _id
        
        return encoder, decoder

    def delete_ones(self) -> dict:
        a = self.df.groupby('userId')['itemId'].size()
        for i in a.index:
            if a[i] <= config['val_data']:
                del(a[i])
        df_ = self.df.copy()
        df_ = df_[df_['userId'].isin(a.index)]
                
        return df_

    def generate_sequence_data(self) -> dict:
        '''
        sequence data 생성

        Return:
            dict: train user sequence / valid user sequence
        '''
        users = defaultdict(list)
        user_train = {}
        user_valid = {}
        group_df = self.df.groupby('user_idx')
        for user, item in group_df:
            users[user].extend(item['item_idx'].tolist())

        for user in users:
            user_train[user] = users[user][:-config['val_data']]
            user_valid[user] = users[user][-config['val_data']:] # 마지막 아이템 예측

        return user_train, user_valid

    def get_train_valid_data(self):
        return self.user_train, self.user_valid

In [6]:
class BERTTrainDataSet(Dataset):
    def __init__(self, user_train, num_users, num_items, sequence_len=200, mask_prob=0.15, random_seed=None):
        self.user_train = user_train
        self.num_users = num_users
        self.num_items = num_items
        self.sequence_len = sequence_len
        self.mask_prob = mask_prob
        self._all_items = set([i for i in range(1, self.num_items + 1)])

        # rng
        if random_seed is None:
            self.rng = random.Random()
        else:
            self.rng = random.Random(random_seed)

        # tokens
        self.mask_token = self.num_items + 1


    def __len__(self):
        # 총 user 수(학습에 사용할 sequence 수수)
        return self.num_users

    def __getitem__(self, user):
        seq = self.user_train[user]
        tokens = []
        labels = []
        for s in seq[-self.sequence_len:]:
            prob = np.random.random()
            if prob < self.mask_prob:
                prob /= self.mask_prob
                # BERT 학습
                # random 하게 80% 를 mask token 으로 변환
                if prob < 0.8:
                    # masking
                    # mask_index : num_item + 1 , 0 : pad, 1 ~ num_item : item index
                    tokens.append(self.mask_token)
                elif prob < 0.9:
                    # item random sampling (noise)
                    tokens.append(np.random.randint(1, self.num_items))
                else:
                    # 나머지 10% 를 original token 으로 사용
                    tokens.append(s)
                labels.append(s) # 학습에 사용
            # training 에 사용하지 않음 
            else:
                tokens.append(s)
                labels.append(0) # 학습에 사용하지 않음, trivial

        # zero padding 
        # tokens 와 labels 가 padding_len 보다 짧으면 zero padding 을 해준다. 
        padding_len = self.sequence_len - len(tokens)
        
        tokens = [0] * padding_len + tokens
        labels = [0] * padding_len + labels
        return torch.LongTensor(tokens), torch.LongTensor(labels)

    def _getseq(self, user):
        return self.user_train[user]

    def random_neg_sampling(self, sold_items, num_item_sample):
        nge_samples = random.sample(list(self._all_items - set(sold_items)), num_item_sample)
        return nge_samples

class BertEvalDataset(Dataset):
    def __init__(self, u2seq, u2answer, sequence_len, mask_token, negative_samples):
        self.u2seq = u2seq
        self.users = sorted(self.u2seq.keys())
        self.u2answer = u2answer
        self.sequence_len = sequence_len
        self.mask_token = mask_token
        self.negative_samples = negative_samples

    def __len__(self):
        return len(self.users)

    def __getitem__(self, index):
        user = self.users[index]
        seq = self.u2seq[user]
        answer = self.u2answer[user]
        negs = self.random_neg_sampling(sold_item=user, num_item_sample = 1)

        candidates = answer + negs
        labels = [1] * len(answer) + [0] * len(negs)

        seq = seq + [self.mask_token]
        seq = seq[-self.sequence_len:]
        padding_len = self.sequence_len - len(seq)
        seq = [0] * padding_len + seq

        return torch.LongTensor(seq), torch.LongTensor(candidates), torch.LongTensor(labels)
    
    def random_neg_sampling(self, sold_items, num_item_sample : int):
        nge_samples = random.sample(list(self._all_items - set(sold_items)), num_item_sample)
        return nge_samples

# 3. 모델

### Util

In [7]:
def fix_random_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

In [8]:
class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

In [9]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.alpha = nn.Parameter(torch.ones(features))
        self.beta = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.alpha * (x - mean) / (std + self.eps) + self.beta

In [10]:
class SublayerConnection(nn.Module):
    # layer가 많아지면 학습이 잘 안되는 현상
    # 따라서, sub-layer들에 각각 dropout을 한 후 서로 residual connection을 적용하고, layer normalization 적용
    def __init__(self, hidden_units, dropout):
        super().__init__()
        self.hidden_units = hidden_units
        self.dropout = dropout

        self.norm = LayerNorm(hidden_units)
        self.dropout = nn.Dropout(dropout)

    def forward(self, sublayer, x):
        "Apply residual connection to any sublayer with the same size."
        r = self.norm(x)
        r = sublayer(r)

        r = self.dropout(r)

        return x + r

In [11]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, hidden_units, d_ff, dropout=0.1):
        super().__init__()

        self.w_1 = nn.Linear(hidden_units, d_ff)
        self.w_2 = nn.Linear(d_ff, hidden_units)
        self.dropout = nn.Dropout(dropout)
        self.activation = GELU()

    def forward(self, x):
        return self.w_2(self.dropout(self.activation(self.w_1(x))))

## Attention

In [12]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super().__init__()
        self.hidden_units = hidden_units
        self.dropout = nn.Dropout(dropout_rate) # dropout rate

    def forward(self, Q, K, V, mask=None, dropout=None):
        """
            b = batch, ? = num_heads, L = sequence_len

            Q: (b x ? x L x dim_Q)
            K: (b x ? x L x dim_K)
            V: (b x ? x L x dim_V)
            ?: 1 (squeezed) or h (multi-head)
            mask: (b x ? x L x L)
            dropout: nn.Module
            assuming dim_Q = dim_K
        """

        # A: (b x ? x L x L)
        attn_score = torch.matmul(Q, K.transpose(2, 3)) / math.sqrt(self.hidden_units)

        # apply mask (the logit value of a padding token should be minus infinity)
        if mask is not None:
            attn_score = attn_score.masked_fill(mask == 0, -1e9) # 유사도가 0인 지점은 -infinity로 보내서 softmax 결과가 0이 아니게 함
        
        # getting normalized(probability) weights through softmax (when padding token, it'll be 0)
        # P: (b x ? x L x L)
        p_attn = F.softmax(attn_score, dim=-1)
        
        # apply dropout (with given dropout)
        if dropout is not None:
            p_attn = self.dropout(p_attn) # attention distribution 상대적 중요도

        # (b x ? x L x L) @ (b x ? x L x dim_V) -> (b x ? x L x dim_V)
        output = torch.matmul(p_attn, V)

        return output, p_attn

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout=0.1):

        """
            dim_V should be equal to hidden_units / num_heads
            we assume dim_Q = dim_K = dim_V
        """

        super().__init__()
        assert hidden_units % num_heads == 0

        self.hidden_units = hidden_units
        self.num_heads = num_heads

        self.dim_V = hidden_units // num_heads

        # query, key, value, output 생성을 위한 Linear 모델 생성
        self.W_Q = nn.Linear(hidden_units, hidden_units * num_heads, bias=False)
        self.W_K = nn.Linear(hidden_units, hidden_units * num_heads, bias=False)
        self.W_V = nn.Linear(hidden_units, hidden_units * num_heads, bias=False)

        self.W_O = nn.Linear(hidden_units * num_heads, hidden_units, bias=False)

        self.attention = ScaledDotProductAttention(hidden_units, dropout)
        self.dropout = nn.Dropout(p=dropout)
        self.layerNorm = LayerNorm(hidden_units)

    def forward(self, enc, mask):

        residual = enc

        batch_size, seqlen = enc.size(0), enc.size(1)

        # 1) Do all the linear projections in a batch from dim_model, then split into (num_heads x dim_V)
        # [process]
        # (1) linear(W): (b x L x dim_model) -> (b x L x dim_model)
        # (2) view: (b x L x dim_model) -> (b x L x num_heads x dim_V)
        # (3) transpose: (b x L x num_heads x dim_V) -> (b x num_heads x L x dim_V)
        # Query, Key, Value를 (num_head)개의 Head로 나누어 각기 다른 Linear projection을 통과시킴
        Q = self.W_Q(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units).transpose(1, 2)
        K = self.W_K(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units).transpose(1, 2)
        V = self.W_V(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units).transpose(1, 2)

        # 2) Apply attention to the projected vectors in the batch
        # note that attenion only cares about the last two dimensions
        # output: (b x num_heads x L x dim_V)
        # Head별로 각기 다른 attention이 가능하도록 각각 attention에 통과시킴
        output, attn_dist = self.attention(Q, K, V, mask=mask, dropout=self.dropout) # attn_dist : (batch_size, num_heads, sequence_len, sequence_len)

        # 3) "concat" those heads using view
        # [process]
        # (1) transpose: (b x num_heads x L x dim_V) -> (b x L x num_heads x dim_V)
        # (2) contiguous: reorder memory inside GPU (no dimension change)
        # (3) view: (b x L x num_heads x dim_V) -> (b x L x dim_model)
        # 다시 Transpose한 후 모든 head들의 attention 결과를 합칩니다.
        output = output.transpose(1, 2).contiguous() # (batch_size, sequence_len, num_heads, d_model) / contiguous() : 가변적 메모리 할당
        output = output.view(batch_size, seqlen, -1) # (batch_size, sequence_len, d_model * num_heads)

        # 4) apply the final linear
        # X: (b x L x dim_model)
        output = self.layerNorm(self.dropout(self.W_O(output)) + residual)

        return output

## Model

In [13]:
class TransformerBlock(nn.Module):
    """
    Bidirectional Encoder = Transformer (self-attention)
    Transformer = MultiHead_Attention + Feed_Forward with sublayer connection
    """

    def __init__(self, hidden_units=256, num_heads=4, d_ff=1024, dropout=0.1):
        """
        :param hidden_units: hidden_units size of transformer
        :param num_heads: head sizes of multi-head attention
        :param feed_forward_hidden: feed_forward_hidden, usually 4*hidden_units
        :param dropout: dropout rate
        """

        super().__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, hidden_units=hidden_units, dropout=dropout)
        self.attention_sublayer = SublayerConnection(hidden_units=hidden_units, dropout=dropout)
        self.pwff = PositionwiseFeedForward(hidden_units=hidden_units, d_ff=d_ff, dropout=dropout)
        self.pwff_sublayer = SublayerConnection(hidden_units=hidden_units, dropout=dropout)
        self.dropoutlayer = nn.Dropout(p=dropout)

    def forward(self, input_enc, mask=None):
        # we need dynamic mask for the attention forward (sublayer module also has parameters, namely layernorm)
        # x: (b x L x dim_model)
        # mask: (b x L x L), set False to ignore that point
        x = self.attention_sublayer(lambda _x: self.attention(_x, mask=mask), input_enc)
        x = self.pwff_sublayer(self.pwff, x)
        return self.dropoutlayer(x)

class BERT4Rec(nn.Module):
    def __init__(self, num_users, num_items, sequence_len, device, num_layers=2, hidden_units=256, num_heads=4, dropout=0.1, random_seed=None):
        super().__init__()

        # params
        self.num_users = num_users
        self.num_items = num_items
        self.sequence_len = sequence_len
        self.num_layers = num_layers
        self.hidden_units = hidden_units
        self.num_heads = num_heads
        self.dropout = dropout
        self.random_seed = random_seed
        self.device = device
        
        # set seed
        if random_seed is not None:
            fix_random_seed(random_seed)

        # 0: padding token
        # 1 ~ V: item tokens
        # V + 1: mask token

        # Embedding layers
        self.item_emb = nn.Embedding(num_items + 2, hidden_units, padding_idx=0) # padding : 0 / item : 1 ~ num_item + 1 /  mask : num_item + 2
        self.pos_emb = nn.Embedding(sequence_len, hidden_units) # learnable positional encoding
        self.dropout = nn.Dropout(dropout)
        self.emb_layernorm = nn.LayerNorm(hidden_units, eps=1e-6)

        # transformer layers
        self.transformers = nn.ModuleList([
            TransformerBlock(
                hidden_units=hidden_units,
                num_heads=num_heads,
                d_ff=hidden_units*4,
                dropout=dropout
            ) for _ in range(num_layers)
        ])
        self.out = nn.Linear(hidden_units, num_items + 1)
    
    def forward(self, tokens):
        L = tokens.shape[1]

        # mask for whether padding token or not in attention matrix (True if padding token)
        # [process] (b x L) -> (b x 1 x L) -> (b x L x L) -> (b x 1 x L x L)
        token_mask = torch.BoolTensor(tokens > 0).unsqueeze(1).repeat(1, L, 1).unsqueeze(1).to(self.device)

        # get embedding
        # [process] (b x L) -> (b x L x d)
        seqs = self.item_emb(torch.LongTensor(tokens).to(self.device))  # (batch_size, sequence_len, hidden_units)
        positions = np.tile(np.array(range(tokens.shape[1])), [tokens.shape[0], 1])  # (batch_size, sequence_len)
        seqs += self.pos_emb(torch.LongTensor(positions).to(self.device))  # (batch_size, sequence_len, hidden_units)
        seqs = self.emb_layernorm(self.dropout(seqs))  # LayerNorm

        # apply multi-layered transformers
        # [process] (b x L x d) -> ... -> (b x L x d)
        for transformer in self.transformers:
            seqs = transformer(seqs, token_mask)

        # classifier
        # [process] (b x L x d) -> (b x L x (V + 1))
        out = self.out(seqs)

        return out

# 4. 학습함수

In [14]:

class Solver:
    def calculate_loss(self, batch):
        
        self.ce_losser = CrossEntropyLoss(ignore_index=0)

        # device
        tokens = batch['tokens'].to(self.device)  # b x L
        labels = batch['labels'].to(self.device)  # b x L

        # forward
        logits = self.model(tokens)  # b x L x (V + 1)

        # loss
        logits = logits.view(-1, logits.size(-1))  # bL x (V + 1)
        labels = labels.view(-1)  # bL
        loss = self.ce_losser(logits, labels)

        return loss

    def set_model_mode(self, purpose: str) -> None:
        if purpose == 'eval':
            self.model = self.model.eval()  # type: ignore
        elif purpose == 'train':
            self.model = self.model.train()

    def solve(self) -> None:
        C = self.config
        name = C['name']
        print(f"solve {name} start")

        # report new session
        self.logger.info("-- new solve session started --")

        # report model parameters
        numels = []
        for parameter in self.model.parameters():
            if parameter.requires_grad:
                numels.append(parameter.numel())
        num_params = sum(numels)
        self.logger.info(f"total parameters:\t{num_params}")
        with open(os.path.join(self.data_dir, 'meta.json'), 'w') as fp:
            json.dump({'num_params': num_params}, fp)

        # solve loop
        self.early_stop = False
        self.patience_counter = 0
        self.load_model('train')
        for epoch in range(self.start_epoch, C['train']['epoch'] + 1):
            self.solve_train(epoch)
            if self.scheduler is not None:
                self.scheduler.step()
            if self.early_stop:
                break

        # solve end
        self.load_model('test')
        self.solve_test()

    def solve_train(self, epoch: int) -> None:
            self.train_one_epoch(epoch)
            self.evaluate_on_valid(epoch)

    def train_one_epoch(self, epoch: int) -> None:
        # prepare
        self.set_model_mode('train')
        epoch_loss_sum = 0.0
        epoch_loss_count = 0

        # loop
        pbar = tqdm(self.train_dataloader)
        pbar.set_description(f"[epoch {epoch} train]")
        for batch in pbar:

            # get loss
            loss = self.calculate_loss(batch)

            # backward
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            # step end
            epoch_loss_sum += float(loss.data)
            epoch_loss_count += 1
            pbar.set_postfix(loss=f'{epoch_loss_sum / epoch_loss_count:.4f}')

        pbar.close()

        # epoch end
        epoch_loss = epoch_loss_sum / epoch_loss_count
        self.logger.info('\t'.join([
            f"[epoch {epoch}]",
            f"train loss: {epoch_loss:.4f}"
        ]))
        self.writer.add_scalar('train_loss', epoch_loss, epoch)

    def evaluate_on_valid(self, epoch: int) -> None:
        self.logger.info(f"<evaluate on valid (epoch: {epoch})>")
        C = self.config

        # prepare
        self.set_model_mode('eval')
        ks = sorted(C['metric']['ks'])
        pivot = C['metric']['pivot']

        # init
        result_values = []

        # loop
        pbar = tqdm(self.valid_dataloader)
        pbar.set_description(f"[epoch {epoch} valid]")
        with torch.no_grad():
            for batch in pbar:

                # get rankers
                rankers = self.calculate_rankers(batch)

                # evaluate
                labels = batch['labels'].to(self.device)
                batch_results = calc_batch_rec_metrics_per_k(rankers, labels, ks)  # type: ignore
                result_values.extend(batch_results['values'][pivot])

                # verbose
                pbar.set_postfix(pivot_score=f'{sum(result_values) / len(result_values):.4f}')

        pbar.close()

        # report
        pivot_score = sum(result_values) / len(result_values)
        self.logger.info('\t'.join([
            f"[epoch {epoch}]",
            f"valid pivot score: {pivot_score:.4f}"
        ]))
        self.writer.add_scalar('valid_pivot_score', pivot_score, epoch)

        # save best
        if self.best_score is None or self.best_score < pivot_score:
            self.best_score = pivot_score
            torch.save(self.model.state_dict(), self.model_path)
            self.logger.info(f"updated best model at epoch {epoch} with pivot score {pivot_score}")
            self.patience_counter = 0
        else:
            self.patience_counter += 1
        if self.patience_counter >= C['train']['patience']:
            self.logger.info(f"no update for {self.patience_counter} epochs, thus early stopping")
            self.early_stop = True

        # save checkpoint
        torch.save({
            'epoch': epoch,
            'best_score': self.best_score,
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
        }, self.check_path)

In [15]:
from torch.nn import CrossEntropyLoss

def train_one_epoch(epoch, train_dataloader):
    model.train()

    epoch_loss_sum = 0.0
    epoch_loss_count = 0

    pbar = tqdm(train_dataloader)
    pbar.set_description(f"[epoch {epoch} train")
    
    for batch in pbar:
        loss = calculate_loss(batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss_sum += float(loss.data)
        epoch_loss_count += 1
        pbar.set_postfix(loss=f'{epoch_loss_sum / epoch_loss_count:.4f}')
    
    pbar.close()

    epoch_loss = epoch_loss_sum / epoch_loss_count

    return epoch_loss

def evaluate_on_valid(epoch, valid_dataloader):
    model.eval()
    
    NDCG = 0.0
    HIT = 0.0

    num_item_sample = 100
    users = [user for user in range(make_sequence_dataset.num_user)]
    
    result_values = []

    pbar = tqdm(valid_dataloader)
    pbar.set_description(f"[epoch {epoch} valid")

    with torch.no_grad():
        for batch in pbar:
            rankers = calculate_rankers(batch)

            labels = batch['labels'].to(config['device'])
            


def train(model, criterion, optimizer, data_loader):
    
    for epoch in range(1, config['num_epochs'] + 1):
        train_loss = train_one_epoch(epoch,data_loader)



    for seq, labels in data_loader:
        logits = model(seq)

        logits = logits.view(-1, logits.size(-1))
        labels = labels.view(-1).to(config['device'])

        optimizer.zero_grad()
        loss = criterion(logits, labels)

        loss_val += loss.item()

        loss.backward()
        optimizer.step()
    
    loss_val /= len(data_loader)

    return loss_val

def evaluate(model, user_train, user_valid, max_len, bert4rec_dataset, make_sequence_dataset):
    model.eval()

    NDCG = 0.0 # NDCG@10
    HIT = 0.0 # HIT@10

    num_item_sample = 100

    users = [user for user in range(make_sequence_dataset.num_user)]

    for user in users:
        seq = (user_train[user] + [make_sequence_dataset.num_item + 1])[-max_len:]
        rated = user_train[user] + user_valid[user]
        items = user_valid[user] + bert4rec_dataset.random_neg_sampling(rated_item = rated, num_item_sample = num_item_sample)
        
        with torch.no_grad():
            predictions = -model(np.array([seq]))
            predictions = predictions[0][-1][items] # sampling
            rank = predictions.argsort().argsort()[0].item()

        if rank < 10: #Top10
            NDCG += 1 / np.log2(rank + 2)
            HIT += 1

    NDCG /= len(users)
    HIT /= len(users)

    return NDCG, HIT

def calculate_loss(batch):
        
    ce_losser = CrossEntropyLoss(ignore_index=0)

    # device
    tokens = batch['tokens'].to(config['device'])  # b x L
    labels = batch['labels'].to(config['device'])  # b x L

    # forward
    logits = model(tokens)  # b x L x (V + 1)

    # loss
    logits = logits.view(-1, logits.size(-1))  # bL x (V + 1)
    labels = labels.view(-1)  # bL
    loss = ce_losser(logits, labels)

    return loss

def calculate_rankers(batch):

    # device
    tokens = batch['tokens'].to(config['device'])  # b x L
    cands = batch['cands'].to(config['device'])  # b x C

    # forward
    logits = model(tokens)  # b x L x (V + 1)

    # gather
    logits = logits[:, -1, :]  # b x (V + 1)
    scores = logits.gather(1, cands)  # b x C
    rankers = scores.argsort(dim=1, descending=True)

    return rankers

In [16]:
def recall(scores, labels, k):
    scores = scores.cpu()
    labels = labels.cpu()
    rank = (-scores).argsort(dim=1)
    cut = rank[:, :k]
    hit = labels.gather(1, cut)
    return (hit.sum(1).float() / labels.sum(1).float()).mean().item()


def ndcg(scores, labels, k):
    scores = scores.cpu()
    labels = labels.cpu()
    rank = (-scores).argsort(dim=1)
    cut = rank[:, :k]
    hits = labels.gather(1, cut)
    position = torch.arange(2, 2+k)
    weights = 1 / torch.log2(position.float())
    dcg = (hits.float() * weights).sum(1)
    idcg = torch.Tensor([weights[:min(n, k)].sum() for n in labels.sum(1)])
    ndcg = dcg / idcg
    return ndcg.mean()


def recalls_and_ndcgs_for_ks(scores, labels, ks):
    metrics = {}

    scores = scores.cpu()
    labels = labels.cpu()
    answer_count = labels.sum(1)
    answer_count_float = answer_count.float()
    labels_float = labels.float()
    rank = (-scores).argsort(dim=1)
    cut = rank
    for k in sorted(ks, reverse=True):
       cut = cut[:, :k]
       hits = labels_float.gather(1, cut)
       metrics['Recall@%d' % k] = (hits.sum(1) / answer_count_float).mean().item()

       position = torch.arange(2, 2+k)
       weights = 1 / torch.log2(position.float())
       dcg = (hits * weights).sum(1)
       idcg = torch.Tensor([weights[:min(n, k)].sum() for n in answer_count])
       ndcg = (dcg / idcg).mean()
       metrics['NDCG@%d' % k] = ndcg

    return metrics

# 5. 학습

In [17]:
make_sequence_dataset = MakeSequenceDataSet(config=config)
user_train, user_valid = make_sequence_dataset.get_train_valid_data()

In [18]:
bert4rec_dataset = BERTTrainDataSet(
    user_train = user_train, 
    sequence_len = config.sequence_len, 
    num_users = make_sequence_dataset.num_users, 
    num_items = make_sequence_dataset.num_items,
    mask_prob = config.mask_prob,
    )

In [19]:
data_loader = DataLoader(
    bert4rec_dataset, 
    batch_size = config.batch_size, 
    shuffle = True, 
    pin_memory = True,
    num_workers = config.num_workers,
    )

In [20]:
model = BERT4Rec(
    num_users = make_sequence_dataset.num_users, 
    num_items = make_sequence_dataset.num_items, 
    hidden_units = config.hidden_units, 
    num_heads = config.num_heads, 
    num_layers = config.num_layers, 
    sequence_len = config.sequence_len, 
    dropout = config.dropout, 
    device = device,
    ).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0) # label이 0인 경우 무시
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

In [21]:
i = 0
for seq, lab in data_loader:
    pred = model(seq)
    print(seq, pred[i], lab, sep='\n')
    break

tensor([[21588, 33743, 44447,  ..., 13223, 37212, 15098],
        [    0,     0,     0,  ...,     0,     0,     0],
        [    0,     0,     0,  ...,     0,     0,     0],
        ...,
        [    0,     0,     0,  ...,     0,     0,     0],
        [    0,     0,     0,  ...,     0,     0,     0],
        [    0,     0,     0,  ..., 44447, 30893, 44447]])
tensor([[-7.3262e-01, -2.6836e+00, -2.0946e+00,  ..., -1.2129e+00, 1.1461e+00, -2.4402e+00],
        [4.8902e-01, -1.1710e+00, 1.2080e-01,  ..., -1.4305e+00, 2.3261e-01, 6.5799e-01],
        [3.9673e-01, 5.0583e-01, -4.1934e+00,  ..., -6.7379e-01, -1.0566e+00, 4.7712e-01],
        ...,
        [2.8523e+00, -4.3828e+00, 8.9948e-01,  ..., 3.0763e+00, -5.9995e-01, -8.0279e-01],
        [1.9858e-01, -3.9231e-01, 3.2045e+00,  ..., -2.0052e+00, 5.6769e-01, 4.8417e-01],
        [-2.0772e+00, 1.5988e+00, -1.9775e+00,  ..., -1.5081e+00, 2.8277e+00, 1.5892e+00]],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[    0,     0,  712

In [22]:
def ndcg(scores, labels, k):
    scores = scores.cpu()
    labels = labels.cpu()
    rank = (-scores).argsort(dim=1)
    cut = rank[:, :k]
    hits = labels.gather(1, cut)
    position = torch.arange(2, 2+k)
    weights = 1 / torch.log2(position.float())
    dcg = (hits.float() * weights).sum(1)
    idcg = torch.Tensor([weights[:min(n, k)].sum() for n in labels.sum(1)])
    ndcg = dcg / idcg
    return ndcg.mean()

In [23]:
lab = torch.as_tensor(lab, device=device)
lab.is_cuda

True

In [24]:
pred.is_cuda

True

In [25]:
pred

tensor([[[-7.3262e-01, -2.6836e+00, -2.0946e+00,  ..., -1.2129e+00,
           1.1461e+00, -2.4402e+00],
         [ 4.8902e-01, -1.1710e+00,  1.2080e-01,  ..., -1.4305e+00,
           2.3261e-01,  6.5799e-01],
         [ 3.9673e-01,  5.0583e-01, -4.1934e+00,  ..., -6.7379e-01,
          -1.0566e+00,  4.7712e-01],
         ...,
         [ 2.8523e+00, -4.3828e+00,  8.9948e-01,  ...,  3.0763e+00,
          -5.9995e-01, -8.0279e-01],
         [ 1.9858e-01, -3.9231e-01,  3.2045e+00,  ..., -2.0052e+00,
           5.6769e-01,  4.8417e-01],
         [-2.0772e+00,  1.5988e+00, -1.9775e+00,  ..., -1.5081e+00,
           2.8277e+00,  1.5892e+00]],

        [[ 6.6500e-01, -5.0477e-02, -1.8718e+00,  ..., -2.4836e+00,
          -2.2910e+00, -3.0228e+00],
         [ 9.6118e-01,  4.1477e+00,  2.3139e+00,  ..., -9.3152e-01,
          -5.9612e-01,  3.1199e+00],
         [ 1.5511e+00,  3.0518e+00, -1.2783e-01,  ..., -2.1445e+00,
          -9.2955e-02,  3.5792e+00],
         ...,
         [ 9.2683e-01, -4

In [26]:
torch.arange(2, 12)

tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [27]:
1 / torch.log2(torch.arange(2, 12).float())

tensor([1.0000e+00, 6.3093e-01, 5.0000e-01, 4.3068e-01, 3.8685e-01, 3.5621e-01, 3.3333e-01, 3.1546e-01, 3.0103e-01,
        2.8906e-01])

In [28]:
(-pred[0]).argsort(dim=1)

tensor([[34203, 12715, 13163,  ..., 39115, 38046, 29882],
        [18105, 16909, 40187,  ...,  2860, 19955,  5684],
        [38471, 22880, 23496,  ...,  7298, 26004, 26732],
        ...,
        [22975, 27121, 35101,  ..., 40363, 12751,  2030],
        [37823, 36953, 19514,  ..., 41071, 41314, 38530],
        [22168,   251, 22184,  ..., 43783, 30912,  6382]], device='cuda:0')

In [30]:
cut[0]

tensor([        10,          0,          0,          0,          0,          1,
                 0,          0,          0,          0], device='cuda:0')

In [31]:
lab[0]

tensor([139810988957344, 139810988957344, 139807760165600,          0,          0,          0,
                 0,          0,          0,          0,          0,          0,
                 0,          0,  124887808,         -1,  123970049,          0,
                 0,          0,          0,          0, 139807760130817,  124864320,
        139808522486913,          0,          0,          0,  124817408,  103567984,
        139807590791504,          0,          0,          0,          1,          3,
               304,        417,  177799376,  124886464,          1,  121589584,
         177795936,          0,  124848768,  103567984, 139807721078592,          0,
                 0,          0], device='cuda:0')

In [29]:
cut = (-pred[0]).argsort(dim=1)[:, :10]
lab[0].gather(0, cut[0])

tensor([124887392, 0, 121661072, 103567984, 139807589665344, 0, 0, 0, 0, 0],
       device='cuda:0')

In [39]:
cut[0]

tensor([         0,          0,          0,          0,          0,          1,
                 0,          0,          0,          0], device='cuda:0')

In [93]:
pred[0]

tensor([[-3.6826e+00, -4.2028e-01, -1.4675e+00,  ..., 5.8835e-01, -2.6691e+00, 3.7967e-01],
        [-1.8520e+00, -5.9354e-01, 1.7134e+00,  ..., -1.3382e+00, 2.5389e+00, 9.2821e-01],
        [3.5316e+00, 1.5960e+00, -2.5019e+00,  ..., 4.9782e-01, -3.0970e+00, 4.7133e-01],
        ...,
        [8.4954e-01, 4.0838e-01, 2.3736e+00,  ..., 9.7804e-02, -3.8567e-01, -4.5163e-01],
        [2.0766e+00, -8.4078e-01, -1.8647e-02,  ..., 1.7163e+00, 4.0223e+00, -8.9374e-01],
        [5.9893e+00, 1.0482e+00, 2.1059e+00,  ..., -2.1189e-02, -3.5698e+00, 1.6093e+00]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [60]:
pred.shape

torch.Size([256, 50, 44447])

In [61]:
pred[:, -1, :].shape

torch.Size([256, 44447])

In [62]:
seq.shape

torch.Size([256, 50])

In [63]:
seq[1]

tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0, 14421, 44447, 30265, 44447, 16811,
         9227, 44447, 44447, 44447, 44447, 44447, 13249,  6750,   552, 44447,
        25394, 30869, 14105, 27263, 32444, 44447,  8826, 44447, 15881, 16941])

In [64]:
lab[1]

tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 18549,     0,  7421,     0,
            0, 20373, 31999, 40806, 16178, 36064,     0,  6750,     0, 24927,
            0,     0,     0,     0,     0, 43540,     0, 31887,     0,     0])

In [88]:
model(np.array(seq))[1][-1][item]

tensor([-2.5180e+00, -4.6483e+00, 2.3331e+00, -2.2213e+00, 1.4559e+00, -3.4876e+00, 2.6763e+00, -1.3567e+00,
        -8.0492e-01, 6.8982e-01, -7.5294e-01, 7.1165e-01, 1.8907e+00, -3.0660e-01, 1.7734e+00, 1.2953e+00,
        -3.0323e+00, 2.4043e+00, -1.8935e+00, -1.6689e+00, -1.7942e-01, 2.2040e+00, -4.8107e+00, -8.8158e-03,
        1.1862e+00, -2.2089e+00, -6.7169e-01, -2.1119e+00, -2.5307e+00, -1.7450e+00, -2.0174e+00, -2.8621e-01,
        -3.1166e+00, 1.5495e+00, -1.7286e-01, 1.0464e+00, -1.4209e+00, -1.8008e+00, -1.8513e+00, -1.4855e+00,
        -2.3269e-01, -6.6396e-01, -2.5133e+00, 1.4452e+00, 1.9394e+00, 1.6085e+00, 7.5975e-01, -1.9973e+00,
        1.0125e+00, -2.8110e+00, 2.9417e+00, -2.9644e-01, -3.8430e+00, -1.0267e+00, 1.2913e+00, 4.2419e-01,
        2.0089e+00, -1.8846e+00, 3.3864e+00, 1.4426e+00, -6.0401e-01, 3.4674e+00, -1.7923e+00, 1.9770e+00,
        -1.4765e+00, 9.8734e-01, 1.7606e+00, -4.6233e+00, 4.8576e+00, 5.2621e+00, -1.9956e-01, 3.7990e-02,
        -1.6113e+00, 1.

In [67]:
pred[1][-1]

tensor([5.3098e-01, 5.8145e-01, -5.6312e-02,  ..., 3.2089e+00, 2.1743e+00, -8.6624e-01],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [73]:
pred[1][-1].max(dim=0).indices.item()

38558

In [81]:
pred[1][-1]

tensor([5.3098e-01, 5.8145e-01, -5.6312e-02,  ..., 3.2089e+00, 2.1743e+00, -8.6624e-01],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [80]:
pred[1][-1].argsort(descending=True)[:10]

tensor([38558, 36962,  9616, 11158, 20558,  5987, 24070, 34526, 40637, 32153],
       device='cuda:0')

In [78]:
pred[1][-1].argsort(descending=True).argsort()[0].item()

17405

In [79]:
pred[1][-1].argsort(descending=True).argsort()

tensor([17405, 16921, 22817,  ...,  1851,  5542, 30106], device='cuda:0')

In [66]:
max(pred[1][-1]).item()

6.816342830657959

In [32]:
lab.shape

torch.Size([256, 50])

In [33]:
len(seq)

256

In [34]:
len(pred)

256

In [35]:
len(pred)

256

In [36]:
len(seq[0])

50

In [37]:
len(lab[0])

50

In [38]:
make_sequence_dataset.num_items

44446

In [39]:
pbar = tqdm(data_loader)

  0%|          | 0/199 [00:00<?, ?it/s]

In [40]:
for b in pbar:
    print(b)
    print(len(b))
    break

  0%|          | 0/199 [00:00<?, ?it/s]

[tensor([[    0,     0,     0,  ...,     0,     0,     0],
        [    0,     0,     0,  ..., 44011, 44447,  1911],
        [    0,     0,     0,  ...,     0,     0,     0],
        ...,
        [ 9612, 34684, 20060,  ..., 35536,  4024,   774],
        [    0,     0,     0,  ..., 34182, 28249, 44447],
        [    0,     0,     0,  ...,     0, 44447, 18931]]), tensor([[    0,     0,     0,  ...,     0,     0,     0],
        [    0,     0,     0,  ...,     0, 16650,     0],
        [    0,     0,     0,  ...,     0,     0,     0],
        ...,
        [    0, 34684, 20060,  ...,     0,     0,     0],
        [    0,     0,     0,  ...,     0,     0,  9289],
        [    0,     0,     0,  ...,     0, 43923,     0]])]
2


In [42]:
print(len(pred.view(-1, pred.size(-1))))

12800


In [43]:
pred.view(-1, pred.size(-1))

tensor([[-4.0563e+00, -1.4749e+00, -1.0410e+00,  ..., -2.2281e-01,
         -1.8372e+00, -6.6261e-03],
        [-5.9312e+00, -1.0652e+00,  2.5878e+00,  ..., -4.2250e-01,
          5.2795e+00, -3.9073e-01],
        [-4.9632e-01,  1.4304e+00, -2.0105e-01,  ..., -2.6102e-01,
         -5.1782e-01, -1.2004e+00],
        ...,
        [ 3.0043e-01,  1.5944e+00,  2.4671e-01,  ..., -7.9227e-01,
          4.3174e-01,  3.0265e-03],
        [ 1.0897e+00,  1.1239e+00, -1.6732e+00,  ...,  1.1516e+00,
         -1.3626e+00, -8.4517e-01],
        [ 2.2963e+00,  2.7915e+00,  1.7088e+00,  ..., -2.6057e+00,
         -7.2375e-01,  1.9439e+00]], device='cuda:0', grad_fn=<ViewBackward0>)

In [44]:
print(len(lab.view(-1)))

12800


In [45]:
lab.view(-1)

tensor([0, 0, 0,  ..., 0, 0, 0])

In [85]:
sold = user_train[900] + user_valid[900]

In [86]:
item = user_valid[900] + bert4rec_dataset.random_neg_sampling(sold_items = sold, num_item_sample = 100)

In [49]:
len(pred[0][-1])

44447

In [50]:
pred = pred[0][-1][item]

In [52]:
len(lab)

256

In [53]:
pred.argsort().argsort()[0].item()

100

In [ ]:
from sklearn.metrics import ndcg_score

with torch.no_grad():
    ndcg_score(lab.cpu().numpy(), pred.cpu().numpy())

In [ ]:
def train(model, criterion, optimizer, data_loader):
    model.train()
    loss_val = 0
    for seq, labels in data_loader:
        logits = model(seq)  # B x T x V

        logits = logits.view(-1, logits.size(-1))  # (B*T) x V
        labels = labels.view(-1).to(device)  # B*T

        optimizer.zero_grad()
        loss = criterion(logits, labels)

        loss_val += loss.item()

        loss.backward()
        optimizer.step()
    
    loss_val /= len(data_loader)

    return loss_val

def evaluate(model, user_train, user_valid, sequence_len, bert4rec_dataset, make_sequence_dataset):
    model.eval()

    NDCG = 0.0 # NDCG@10
    HIT = 0.0 # HIT@10

    num_item_sample = 100

    users = [user for user in range(make_sequence_dataset.num_users)]

    for user in users:
        seq = (user_train[user] + [make_sequence_dataset.num_items + 1] * config['val_data'])[-sequence_len:]
        sold = user_train[user] + user_valid[user]
        items = user_valid[user] + bert4rec_dataset.random_neg_sampling(sold_items = sold, num_item_sample = num_item_sample)
        
        with torch.no_grad():
            predictions = model(np.array([seq]))
            predictions = predictions[0][-1][items] # sampling
            rank = predictions.argsort().argsort()[0].item()

        for i, r in enumerate(predictions):
        if rank < 10: #Top10
            NDCG += 1 / np.log2(rank + 2)
            HIT += 1

    NDCG /= len(users)
    HIT /= len(users)

    return NDCG, HIT

In [ ]:
loss_list = []
ndcg_list = []
hit_list = []
for epoch in range(1, config['num_epochs'] + 1):
    model.train()
    tbar = tqdm(data_loader)
    for _ in tbar:
        train_loss = train(
            model = model, 
            criterion = criterion, 
            optimizer = optimizer, 
            data_loader = data_loader)
        
        ndcg, hit = evaluate(
            model = model, 
            user_train = user_train, 
            user_valid = user_valid, 
            sequence_len = config.sequence_len,
            bert4rec_dataset = bert4rec_dataset, 
            make_sequence_dataset = make_sequence_dataset,
            )

        loss_list.append(train_loss)
        ndcg_list.append(ndcg)
        hit_list.append(hit)

        tbar.set_description(f'Epoch: {epoch:3d}| Train loss: {train_loss:.5f}| NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 3, figsize = (15, 5))
ax = ax.flatten()
epochs = [i for i in range(1, config.num_epochs + 1)]

ax[0].plot(epochs, loss_list)
ax[0].set_title('Loss')

ax[1].plot(epochs, ndcg_list)
ax[1].set_title('NDCG')

ax[2].plot(epochs, hit_list)
ax[2].set_title('HIT')
plt.show()
     